# Experiment: $EXPERIMENT NAME
## Last Updated $DATE -  $AUTHOR.

```
Summary of High Level Research Question
```

Try to scope your experiments such you can answer your research question in 1-3 hours.
This is an ideal time block to enter flow / deep work, but short enough that you will still feel 
motivated by a relatively tight feedback loop.

If a problem seems like it needs more time that that, 

### High Level Experiment Design

## Goals:
```
List of specific goals that this experiment seeks to achieve.

This should fall under a few categories:
- Development of Intuition about a _specific_ topic
- Novel Research or Insight that could lead to a publishable result
- Meaningfully explore a topic which could lead to an improvement in product

Guiding principles should understanding, insight, and value creation.
```

## Tasks & Experiment Design

```
A list of specific tasks that are going to be tested 

```


## Outcomes

```
Document high level research findings and how
```


In [1]:
# %pip install 
# %pip install circuitsvis
# %pip install plotly

In [2]:
# Generic Set of Imports for MI Research
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [3]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [4]:
# Setup PyTorch configuration for inference based experiments
# NOTE: Mark as False if you want to do any kind of training 
#       as part of your experimentation

INFERENCE_ONLY_EXPERIMENT = True
if INFERENCE_ONLY_EXPERIMENT:
    torch.set_grad_enabled(False)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

import plotly.io as pio
pio.renderers.default = "notebook_connected"

cuda


In [5]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [6]:
# Load Circuit Visualizations
# TODO: Explore building out our own packages / tooling
import circuitsvis as cv
# Testing that the library works
cv.examples.hello("Vivek")


In [7]:
# Load & Run a Model
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-7b")
hf_model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-7b")

print("Loaded hf_model, hooking transformer into TransformerLens!")
# model = HookedTransformer.from_pretrained(
#     "EleutherAI/pythia-6.9b-deduped",
#     center_unembed=False,
#     center_writing_weights=False,
#     fold_ln=False,
#     refactor_factored_attn_matrices=True,
#     hf_model=hf_model
# )

### Janky Shit
### TODO: Figure out how this library actually works and make this a cleaner integration.
import transformer_lens.loading_from_pretrained as loading
# Get the model name used in HuggingFace, rather than the alias.
official_model_name = loading.get_official_model_name("EleutherAI/pythia-6.9b-deduped")


# Load the config into an HookedTransformerConfig object. If loading from a
# checkpoint, the config object will contain the information about the
# checkpoint
cfg = loading.get_pretrained_model_config(
    official_model_name,
    checkpoint_index=None,
    checkpoint_value=None,
    fold_ln=False,
    device=device,
    n_devices=1,
)
print(cfg)
cfg.d_vocab = 50280
cfg.d_vocab_out = 50280
print(cfg)


# Get the state dict of the model (ie a mapping of parameter names to tensors), processed to match the HookedTransformer parameter names.
state_dict = loading.get_pretrained_state_dict(
    official_model_name, cfg, hf_model
)

# Create the HookedTransformer object
model = HookedTransformer(cfg, tokenizer=tokenizer)

model.load_and_process_state_dict(
    state_dict,
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    refactor_factored_attn_matrices=False,
    move_state_dict_to_device=True,
)

print(f"Loaded pretrained model into HookedTransformer!")

model_description_text = """For this demo notebook we'll look at Dolly v2. It is based on pythia 6.9b, but we use the weights for dolly v2. To try the model the model out, let's find the loss on this paragraph!"""
# return_type of model can be loss, logits, both, or none!
loss = model(model_description_text, return_type="loss")
print("Model loss:", loss)


Loaded hf_model, hooking transformer into TransformerLens!
HookedTransformerConfig:
{'act_fn': 'gelu',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 128,
 'd_mlp': 16384,
 'd_model': 4096,
 'd_vocab': 50432,
 'd_vocab_out': 50432,
 'device': 'cuda',
 'eps': 1e-05,
 'final_rms': False,
 'from_checkpoint': False,
 'gated_mlp': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.0125,
 'model_name': 'pythia-6.9b-deduped',
 'n_ctx': 2048,
 'n_devices': 1,
 'n_heads': 32,
 'n_layers': 32,
 'n_params': 6442450944,
 'normalization_type': 'LN',
 'original_architecture': 'GPTNeoXForCausalLM',
 'parallel_attn_mlp': True,
 'positional_embedding_type': 'rotary',
 'rotary_dim': 32,
 'scale_attn_by_inverse_layer_idx': False,
 'seed': None,
 'tokenizer_name': 'EleutherAI/pythia-6.9b-deduped',
 'use_attn_result': False,
 'use_attn_scale': True,
 'use_hook_to

In [23]:
# Tokenization
example_prompt = "the founder of Facebook is Mark"
example_answer = "Zuckerberg"

utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'the', ' founder', ' of', ' Facebook', ' is', ' Mark']
Tokenized answer: [' Z', 'ucker', 'berg']


Performance on answer token:
Rank: 0        Logit: 22.91 Prob: 99.94% Token: | Z|

Top 0th token. Logit: 22.91 Prob: 99.94% Token: | Z|
Top 1th token. Logit: 15.02 Prob:  0.04% Token: | z|
Top 2th token. Logit: 12.46 Prob:  0.00% Token: | E|
Top 3th token. Logit: 12.16 Prob:  0.00% Token: | Elliot|
Top 4th token. Logit: 11.97 Prob:  0.00% Token: |
|
Top 5th token. Logit: 11.81 Prob:  0.00% Token: | Cuban|
Top 6th token. Logit: 11.52 Prob:  0.00% Token: |Z|
Top 7th token. Logit: 11.40 Prob:  0.00% Token: |  |
Top 8th token. Logit: 10.68 Prob:  0.00% Token: |us|
Top 9th token. Logit: 10.56 Prob:  0.00% Token: |.|


Performance on answer token:
Rank: 0        Logit: 23.77 Prob: 99.60% Token: |ucker|

Top 0th token. Logit: 23.77 Prob: 99.60% Token: |ucker|
Top 1th token. Logit: 17.93 Prob:  0.29% Token: |uk|
Top 2th token. Logit: 16.55 Prob:  0.07% Token: |uck|
Top 3th token. Logit: 15.53 Prob:  0.03% Token: |uc|
Top 4th token. Logit: 13.66 Prob:  0.00% Token: |UCK|
Top 5th token. Logit: 12.91 Prob:  0.00% Token: |.|
Top 6th token. Logit: 11.87 Prob:  0.00% Token: |ub|
Top 7th token. Logit: 11.50 Prob:  0.00% Token: |ucks|
Top 8th token. Logit: 11.00 Prob:  0.00% Token: |ander|
Top 9th token. Logit: 10.94 Prob:  0.00% Token: |im|


Performance on answer token:
Rank: 0        Logit: 25.24 Prob: 98.27% Token: |berg|

Top 0th token. Logit: 25.24 Prob: 98.27% Token: |berg|
Top 1th token. Logit: 21.15 Prob:  1.65% Token: |burg|
Top 2th token. Logit: 17.57 Prob:  0.05% Token: |ber|
Top 3th token. Logit: 16.38 Prob:  0.01% Token: |borg|
Top 4th token. Logit: 15.81 Prob:  0.01% Token: |beg|
Top 5th token. Logit: 13.86 Prob:  0.00% Token: |bert|
Top 6th token. Logit: 13.72 Prob:  0.00% Token: |bur|
Top 7th token. Logit: 13.15 Prob:  0.00% Token: |­|
Top 8th token. Logit: 12.94 Prob:  0.00% Token: |b|
Top 9th token. Logit: 12.89 Prob:  0.00% Token: |berger|


Ranks of the answer tokens: [(' Z', 0), ('ucker', 0), ('berg', 0)]

In [9]:

tokens = model.to_tokens(example_prompt, prepend_bos=True)
# Move the tokens to the GPU
tokens = tokens.cuda()
# Run the model and cache all activations
original_logits, cache = model.run_with_cache(tokens)


# Different Strategies for Decoding Text

## Greedy

## Sampling

## Top K Sampling

## Nucleus Sampling




In [10]:
def logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):
    # Only the final logits are relevant for the answer
    final_logits = logits[:, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
    answer_logit_diff = answer_logits[:, 0] - answer_logits[:, 1]
    if per_prompt:
        return answer_logit_diff
    else:
        return answer_logit_diff.mean()

print("Per prompt logit difference:", logits_to_ave_logit_diff(original_logits, answer_tokens, per_prompt=True))
original_average_logit_diff = logits_to_ave_logit_diff(original_logits, answer_tokens)
print("Average logit difference:", logits_to_ave_logit_diff(original_logits, answer_tokens).item())

NameError: name 'answer_tokens' is not defined

In [ ]:
# Ablate a Layer

In [ ]:
# Ablate an Attention Head

In [ ]:
# Ablate an Neuron
LAYER_TO_ABLATE = 0
MLP_NEURON_TO_ABLATE = 0



In [12]:
model.generate("On halloween, all the children go Trick or", 
               temperature=0,)

  0%|          | 0/10 [00:00<?, ?it/s]

'On halloween, all the children go Trick or Treating. My son is not a fan of'